In [1]:
import requests
import json

In [9]:
API_KEY = '40d34a48ad1e319afa1275787cab0c61'
baseurl = 'https://api.the-odds-api.com'
sport = 'americanfootball_nfl'
regions = 'us,eu'
oddsFormat = 'american'
markets = 'h2h'
getsports_url = f'/v4/sports/{sport}/odds?regions={regions}&oddsFormat={oddsFormat}&markets={markets}&apiKey={API_KEY}'
response = json.loads(requests.request(method='GET',url=baseurl+getsports_url).text)

In [10]:
outer_list = []
book_list = ['FanDuel','draftkings','betmgm','wynnbet','pinnacle','betrivers','twinspires','barstool','pointsbetus','williamhill_us']
for resp in response:
    home_team = resp['home_team']
    away_team = resp['away_team']
    common_keys = ({
        'id': resp['id'],
        'sport_title': resp['sport_title'],
        'home_team': resp['home_team'],
        'away_team': resp['away_team'],
        'bookmaker': 'barstool',
        'odds_home': 350,
        'odds_away': -200
    })
    for book in resp['bookmakers']:
        if book['key'] in book_list:
            row = common_keys.copy()
            row['bookmaker'] = book['key']
            row['odds_home'] = book['markets'][0]['outcomes'][0]['price'] if book['markets'][0]['outcomes'][0]['name'] == home_team else book['markets'][0]['outcomes'][1]['price']
            row['odds_away'] = book['markets'][0]['outcomes'][1]['price'] if book['markets'][0]['outcomes'][1]['name'] == away_team else book['markets'][0]['outcomes'][0]['price']
            row['dog_odds'] = max(row['odds_home'],row['odds_away'])
            row['fav_odds'] = min(row['odds_home'],row['odds_away'])
            row['dog_team'] = home_team if row['odds_home'] > row['odds_away'] else away_team
            row['fav_team'] = home_team if row['odds_home'] < row['odds_away'] else away_team
            outer_list.append(row)



In [20]:
#response[0]

In [35]:
import pandas as pd
import numpy as np
main = pd.DataFrame(outer_list)
main.head()

,id,sport_title,home_team,away_team,bookmaker,odds_home,odds_away,dog_odds,fav_odds,dog_team,fav_team
0,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,williamhill_us,200,-240,200,-240,Chicago Bears,Baltimore Ravens
1,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,betrivers,190,-220,190,-220,Chicago Bears,Baltimore Ravens
2,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,barstool,185,-230,185,-230,Chicago Bears,Baltimore Ravens
3,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,twinspires,185,-230,185,-230,Chicago Bears,Baltimore Ravens
4,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,pinnacle,200,-227,200,-227,Chicago Bears,Baltimore Ravens


In [36]:
from bonusOptimization import HelperFunctions, BonusOptimizer
pinnacle = main[main.bookmaker == 'pinnacle'].copy()
pinnacle['dog_fairodds'] = pinnacle.apply(lambda x: HelperFunctions.fairOdds(x.dog_odds,x.fav_odds)['odds1'],axis=1)
pinnacle['fav_fairodds'] = pinnacle.apply(lambda x: HelperFunctions.fairOdds(x.dog_odds,x.fav_odds)['odds2'],axis=1)
pinnacle.head()

,id,sport_title,home_team,away_team,bookmaker,odds_home,odds_away,dog_odds,fav_odds,dog_team,fav_team,dog_fairodds,fav_fairodds
4,a1539d3a46bd8a25219e57e9ff47ce67,NFL,Chicago Bears,Baltimore Ravens,pinnacle,200,-227,200,-227,Chicago Bears,Baltimore Ravens,212.918660,-212.918660
13,28a5d7f9583e72c41f9dc64b356c9745,NFL,Buffalo Bills,Indianapolis Colts,pinnacle,-308,265,265,-308,Indianapolis Colts,Buffalo Bills,285.304010,-285.304010
22,895125b9cabf2adc0cea4a41d3160e9e,NFL,Carolina Panthers,Washington Football Team,pinnacle,-168,151,151,-168,Washington Football Team,Carolina Panthers,159.221580,-159.221580
35,42d9f696dd9159726c0ff6f638847fb1,NFL,Cleveland Browns,Detroit Lions,pinnacle,-645,502,502,-645,Detroit Lions,Cleveland Browns,565.909428,-565.909428
40,2ad0aeaa104a2d396adf41a399a60527,NFL,Minnesota Vikings,Green Bay Packers,pinnacle,104,-114,104,-114,Minnesota Vikings,Green Bay Packers,108.880383,-108.880383


In [44]:
def get_dogFairOdds(x):
    if x.id in pinnacle.id.values:
        dog_fairodds = float(pinnacle[pinnacle.id==x.id].dog_fairodds.values)
    else:
        dog_fairodds = np.nan
    return dog_fairodds
def get_favFairOdds(x):
    if x.id in pinnacle.id.values:
        fav_fairodds = float(pinnacle[pinnacle.id == x.id].fav_fairodds.values)
    else:
        fav_fairodds = np.nan
    return fav_fairodds
main['dog_fairodds'] = main.apply(lambda x: get_dogFairOdds(x),axis=1)
main['fav_fairodds'] = main.apply(lambda x: get_favFairOdds(x),axis=1)
main.dropna()
main['dog_ev'] = main.apply(lambda x: HelperFunctions.EVgivenFairOdds(x.dog_odds,x.dog_fairodds),axis=1)
main['fav_ev'] = main.apply(lambda x: HelperFunctions.EVgivenFairOdds(x.fav_odds,x.fav_fairodds),axis=1)
main.sort_values('fav_ev',ascending=False).head(10)

,id,sport_title,home_team,away_team,bookmaker,odds_home,odds_away,dog_odds,fav_odds,dog_team,fav_team,dog_fairodds,fav_fairodds,dog_ev,fav_ev
88,7ef6f79bc34e5a7f3084484987ad8031,NFL,Las Vegas Raiders,Cincinnati Bengals,betmgm,-105,-115,-105,-115,Las Vegas Raiders,Cincinnati Bengals,117.834862,-117.834862,-0.103733,0.011316
108,a9194e2cd776f2798d030a1968261995,NFL,Los Angeles Chargers,Pittsburgh Steelers,betrivers,-230,200,200,-230,Pittsburgh Steelers,Los Angeles Chargers,237.242604,-237.242604,-0.110433,0.009337
32,42d9f696dd9159726c0ff6f638847fb1,NFL,Cleveland Browns,Detroit Lions,betmgm,-550,400,400,-550,Detroit Lions,Cleveland Browns,565.909428,-565.909428,-0.249147,0.004344
33,42d9f696dd9159726c0ff6f638847fb1,NFL,Cleveland Browns,Detroit Lions,wynnbet,-556,425,425,-556,Detroit Lions,Cleveland Browns,565.909428,-565.909428,-0.211604,0.002676
24,895125b9cabf2adc0cea4a41d3160e9e,NFL,Carolina Panthers,Washington Football Team,betmgm,-160,135,135,-160,Washington Football Team,Carolina Panthers,159.221580,-159.221580,-0.093440,-0.001877
48,f3e4b79c23e4dba39217ee23576bbbac,NFL,Tennessee Titans,Houston Texans,betrivers,-420,350,350,-420,Houston Texans,Tennessee Titans,414.171981,-414.171981,-0.124806,-0.002699
110,a9194e2cd776f2798d030a1968261995,NFL,Los Angeles Chargers,Pittsburgh Steelers,barstool,-240,195,195,-240,Pittsburgh Steelers,Los Angeles Chargers,237.242604,-237.242604,-0.125259,-0.003407
107,a9194e2cd776f2798d030a1968261995,NFL,Los Angeles Chargers,Pittsburgh Steelers,twinspires,-240,195,195,-240,Pittsburgh Steelers,Los Angeles Chargers,237.242604,-237.242604,-0.125259,-0.003407
59,f615a7239e5348cf52bd41cfea0fd3b1,NFL,Jacksonville Jaguars,San Francisco 49ers,betmgm,230,-275,230,-275,Jacksonville Jaguars,San Francisco 49ers,268.470907,-268.470907,-0.104407,-0.006443
112,a9194e2cd776f2798d030a1968261995,NFL,Los Angeles Chargers,Pittsburgh Steelers,wynnbet,-244,205,205,-244,Pittsburgh Steelers,Los Angeles Chargers,237.242604,-237.242604,-0.095607,-0.008212


In [38]:
rows_final = []
for row in outer_list:
    row1 = ({
        'game_id': row['id'],
        'game_name': f"{row['away_team']} @ {row['home_team']}",
        'sport_title': row['sport_title'],
        'bet_name': f"{row['dog_team']} {row['dog_odds']}",
        'odds': row['dog_odds'],
        'bookmaker': row['bookmaker']
    })
    row2 = ({
        'game_id': row['id'],
        'game_name': f"{row['away_team']} @ {row['home_team']}",
        'sport_title': row['sport_title'],
        'bet_name': f"{row['fav_team']} {row['fav_odds']}",
        'odds': row['fav_odds'],
        'bookmaker': row['bookmaker']
    })
    rows_final.append(row1)
    rows_final.append(row2)

In [39]:
float(pinnacle[pinnacle.id == 'a1539d3a46bd8a25219e57e9ff47ce67'].dog_fairodds.values)

212.9186602870814